In [1]:
import sys
import soundcloud

In [2]:
from functools import partial

In [3]:
def getUserAttr(resource, attr):
#   if hasattr(resource, 'user'): return resource.user[attr]
    if hasattr(resource, attr): return getattr(resource, attr)
    return None

getUsername = partial(getUserAttr, attr='username')
getUserid = partial(getUserAttr, attr='id')

In [5]:
# A global artist graph used to iterate through the various algorithms.
# Each node is artist id, with edges weighted by activity between then.
#profileGraph = nx.MultiDiGraph()

client = soundcloud.Client(client_id='454aeaee30d3533d6d8f448556b50f23')

name = input("Enter a soundcloud artist to analyze: ")


Enter a soundcloud artist to analyze: sybyr


In [7]:
# Artist of interest
search = client.get('/users/', q = name)[0]

In [8]:
search

In [12]:
search.__dict__

{'obj': {'avatar_url': 'https://i1.sndcdn.com/avatars-cIS3ELGYY7j2e0q9-OEySHA-large.jpg',
  'id': 13377746,
  'kind': 'user',
  'permalink_url': 'http://soundcloud.com/sybyr',
  'uri': 'https://api.soundcloud.com/users/13377746',
  'username': 'Sybyr',
  'permalink': 'sybyr',
  'last_modified': '2020/07/09 03:24:07 +0000',
  'first_name': 'Sybyrmusic.com',
  'last_name': '',
  'full_name': 'Sybyrmusic.com',
  'city': '',
  'description': 'bookings/email: sybyrcentral@gmail.com\nAnti-World: http://antiworld.co (@antiworldglobe) on all networks\n(Prod. by Landfill) at http://prodbylandfill.com',
  'country': 'United States',
  'track_count': 435,
  'public_favorites_count': 0,
  'followers_count': 51673,
  'followings_count': 22,
  'plan': 'Pro Unlimited',
  'myspace_name': None,
  'discogs_name': None,
  'website_title': 'click Here',
  'website': 'http://antiworld.lnk.to/SYBYR',
  'reposts_count': 99,
  'comments_count': 110,
  'online': False,
  'likes_count': 0,
  'playlist_count': 3

In [15]:
print("Artist interpreted as: %s" % getUsername(search))

Artist interpreted as: Sybyr
